In [1]:
import pandas as pd
import numpy  as np

import polars as pl

In [22]:
results = pd.read_parquet('../data/raw/pl_game_results.parquet')

In [5]:
results = pl.read_parquet('../data/raw/pl_game_results.parquet')

In [3]:
results2 = results

In [6]:
results

season,round,venue,date,home_team_name,home_team_points,away_team_name,away_team_points
i64,i64,str,date,str,i16,str,i16
2018,25,"""Stadium Austra...",2018-08-30,"""Rabbitohs""",51,"""Wests Tigers""",10
2018,25,"""Mt Smart Stadi...",2018-08-31,"""Warriors""",20,"""Raiders""",16
2018,25,"""AAMI Park, Mel...",2018-08-31,"""Storm""",16,"""Panthers""",22
2018,25,"""McDonald Jones...",2018-09-01,"""Knights""",14,"""Dragons""",24
2018,25,"""Cbus Super Sta...",2018-09-01,"""Titans""",26,"""Cowboys""",30
2018,25,"""Stadium Austra...",2018-09-01,"""Eels""",10,"""Roosters""",44
2018,25,"""Stadium Austra...",2018-09-02,"""Bulldogs""",18,"""Sharks""",30
2018,25,"""Suncorp Stadiu...",2018-09-02,"""Broncos""",48,"""Sea Eagles""",16
2018,24,"""Campbelltown S...",2018-08-23,"""Wests Tigers""",22,"""Sea Eagles""",20


In [26]:
results = results.with_columns(pl.col("home_team_points").cast(pl.Int64))

In [7]:
rdf = results 

all_results

In [86]:
def result_expander(rdf):
    '''
    Duplicate results table such that each game has a row where each team is prime and opt.

    This creates a dataframe that can be used for creating lagged effects on each team's performance.
    '''
    results_home = rdf.clone()
    results_away = rdf.clone()
    results_home = results_home.with_columns(home_away=pl.lit('home'))
    results_away = results_away.with_columns(home_away=pl.lit('away'))

    home_cols = ['season', 'round', 'venue', 'date', 'prime_team', 'prime_team_points',
                    'opt_team', 'opt_team_points', 'home_away']
    away_cols = ['season', 'round', 'venue', 'date', 'opt_team', 'opt_team_points',
                    'prime_team', 'prime_team_points', 'home_away']
    results_home.columns = home_cols
    results_away.columns = away_cols

    results_home = results_home.select(home_cols)
    results_away = results_away.select(home_cols) # this looks weird but is intended to select these columns so that the two dataframes can be concatenated

    all_results = pl.concat([results_home, results_away])
    all_results = all_results.with_columns(prime_margin=pl.col('prime_team_points') - pl.col('opt_team_points'))
    all_results=all_results.with_columns(result=np.sign(pl.col('prime_margin')))
    #all_results = all_results.reset_index(drop=True)

        ## make categorical things categorical types
    all_results=all_results.with_columns(home_away=pl.col('home_away').cast(pl.Categorical))
    all_results=all_results.with_columns(venue=pl.col('venue').cast(pl.Categorical))
    all_results=all_results.with_columns(prime_team=pl.col('prime_team').cast(pl.Categorical))
    all_results=all_results.with_columns(opt_team=pl.col('opt_team').cast(pl.Categorical))
    return all_results

    
def create_features(erdf):
    erdf = erdf.sort('prime_team', 'date')
    lagged_cols = [f'prime_margin_{i}' for i in range(1,7)]
    lags = erdf.groupby('prime_team').agg([pl.col('prime_margin').shift(i).alias(f"prime_margin_{i}") for i in range(1,7)]).explode(lagged_cols)
    lags = lags.select(*lagged_cols)
    l6wm = lags.mean(axis=1).alias('pm_l6wm').to_frame()
    erdf = pl.concat([erdf, lags, l6wm], how='horizontal')

    return erdf

In [87]:
expanded_results = result_expander(results)
feature_df = create_features(expanded_results)

In [88]:
feature_df.filter(pl.col('result')==0)

season,round,venue,date,prime_team,prime_team_points,opt_team,opt_team_points,home_away,prime_margin,result,prime_margin_1,prime_margin_2,prime_margin_3,prime_margin_4,prime_margin_5,prime_margin_6,pm_l6wm
i64,i64,cat,date,cat,i16,cat,i16,cat,i16,i16,i16,i16,i16,i16,i16,i16,f64
2019,17,"""Suncorp Stadiu...",2019-07-13,"""Warriors""",18,"""Broncos""",18,"""away""",0,0,4,-1,4,-22,-6,20,-0.166667
2020,3,"""Campbelltown S...",2020-05-31,"""Knights""",14,"""Panthers""",14,"""away""",0,0,18,20,-44,34,-42,36,3.666667
2019,17,"""Suncorp Stadiu...",2019-07-13,"""Broncos""",18,"""Warriors""",18,"""home""",0,0,2,-14,-28,-8,6,5,-6.166667
2020,3,"""Campbelltown S...",2020-05-31,"""Panthers""",14,"""Knights""",14,"""home""",0,0,4,6,44,-16,-14,-12,2.0


In [89]:
feature_df.write_parquet('../data/raw/features.parquet')

In [90]:
feature_df.filter(pl.col('season') == 2023)

season,round,venue,date,prime_team,prime_team_points,opt_team,opt_team_points,home_away,prime_margin,result,prime_margin_1,prime_margin_2,prime_margin_3,prime_margin_4,prime_margin_5,prime_margin_6,pm_l6wm
i64,i64,cat,date,cat,i16,cat,i16,cat,i16,i16,i16,i16,i16,i16,i16,i16,f64
2023,1,"""PointsBet Stad...",2023-03-04,"""Rabbitohs""",27,"""Sharks""",18,"""away""",9,1,-10,10,-4,26,38,-1,9.833333
2023,2,"""BlueBet Stadiu...",2023-03-09,"""Rabbitohs""",10,"""Panthers""",16,"""away""",-6,-1,9,-10,10,-4,26,38,11.5
2023,3,"""Allianz Stadiu...",2023-03-17,"""Rabbitohs""",null,"""Roosters""",null,"""away""",null,null,-6,9,-10,10,-4,26,4.166667
2023,4,"""Accor Stadium,...",2023-03-25,"""Rabbitohs""",null,"""Sea Eagles""",null,"""home""",null,null,null,-6,9,-10,10,-4,-0.2
2023,5,"""Accor Stadium,...",2023-03-31,"""Rabbitohs""",null,"""Storm""",null,"""home""",null,null,null,null,-6,9,-10,10,0.75
2023,6,"""Accor Stadium,...",2023-04-07,"""Rabbitohs""",null,"""Bulldogs""",null,"""away""",null,null,null,null,null,-6,9,-10,-2.333333
2023,7,"""Suncorp Stadiu...",2023-04-13,"""Rabbitohs""",null,"""Dolphins""",null,"""away""",null,null,null,null,null,null,-6,9,1.5
2023,8,"""Accor Stadium,...",2023-04-20,"""Rabbitohs""",null,"""Panthers""",null,"""home""",null,null,null,null,null,null,null,-6,-6.0
2023,9,"""Suncorp Stadiu...",2023-04-28,"""Rabbitohs""",null,"""Broncos""",null,"""away""",null,null,null,null,null,null,null,null,null
